# Counterfactuals Membership Inference Experiment

In [1]:
import pandas as pd
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn.ensemble as es
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
import logging
import warnings
import dice_ml

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
threads = 31

logging.basicConfig()

logger = logging.getLogger('xai-privacy')

In [3]:
from experiment_setup import run_all_experiments
from experiment_setup import get_heart_disease_dataset
from experiment_setup import get_census_dataset
from experiment_setup import MembershipInference

In [4]:
DATASET_HALF = False

data_heart_dict, data_heart_num_dict, data_heart_cat_dict = get_heart_disease_dataset(halve_dataset=DATASET_HALF)
data_census_dict, data_census_num_dict, data_census_cat_dict = get_census_dataset(halve_dataset=DATASET_HALF)

data_heart = data_heart_dict['dataset']
outcome_name_heart = data_heart_dict['outcome']
numeric_features_heart = data_heart_dict['num']

Feature Age: removed 0 rows for missing values.
Feature RestingBP: removed 59 rows for missing values.
Feature Cholesterol: removed 27 rows for missing values.
Feature FastingBS: add unknown category 2.0
Feature RestingECG: add unknown category 3.0
Feature MaxHR: removed 0 rows for missing values.
Feature Oldpeak: removed 7 rows for missing values.
Feature ST_Slope: add unknown category 4.0
Feature CA: add unknown category 4.0
Feature Thal: add unknown category 8.0
Dropped 271 of 1097
Dropped 273 of 1097
Dropped 277 of 1097
Dropped: 2399 of 32561
census: Dropped 3848 of 30162
num: Dropped 19859 of 30162
cat: Dropped 12136 of 30162


This notebook will test whether membership inference is possible with counterfactuals (CF) that are drawn from the training data. Membership inference means an attacker with access to the explanation can determine for any sample whether it was included in the training data or not.

The idea for counterfactual membership inference is as follows: The attacker obtains counterfactuals for the given sample ("frist counterfactuals"). They access the explainer again to receive counterfactuals for each first counterfactual ("second counterfactuals"). The second counterfactuals should have the same class as the original given sample (because the class was flipped twice). If any second counterfactual is equal to the given sample, then the sample must be part of the training data. This concept relies on the assumption that there is a high likelihood that a sample from the training data will be its own counter-counterfactual.

First, we implement the `train_explainer` and `membership_inference_attack_no_model_access` functions:

In [5]:
# Attack code must be imported so that multiprocessing pool works. Check out cf_attack.py for the implementation of the attack.
from cf_attack import CounterfactualMembershipInference

# Executing Membership Inference

We now generate five counterfactuals for the first sample from the training data to demonstrate counterfactual explanations in general.

In [6]:
features = data_heart.drop(outcome_name_heart, axis=1)
labels = data_heart[outcome_name_heart]

# Train a random forest on training data.
model = es.RandomForestClassifier(random_state=0)
model = model.fit(features, labels)

# Train explainer
d = dice_ml.Data(dataframe=data_heart, continuous_features=numeric_features_heart, outcome_name=outcome_name_heart)


m = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')
# Generating counterfactuals from training data (kd-tree)
exp = dice_ml.Dice(d, m, method="kdtree")

In [7]:
e1 = exp.generate_counterfactuals(features[0:1], total_CFs=5, desired_class="opposite")
e1.visualize_as_dataframe(display_sparse_df=False)

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

Query instance (original outcome : 0)


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal,HeartDisease
0,47.0,1.0,1.0,110.0,249.0,0.0,0.0,150.0,0.0,0.0,4.0,4.0,8.0,0.0



Diverse Counterfactual set without sparsity correction (new outcome:  1


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,CA,Thal
388,35.0,1.0,2.0,110.0,257.0,0.0,0.0,140.0,0.0,0.0,4.0,4.0,8.0
434,47.0,1.0,3.0,108.0,243.0,0.0,0.0,152.0,0.0,0.0,1.0,0.0,3.0
705,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,4.0,4.0,7.0
320,46.0,1.0,4.0,120.0,249.0,0.0,2.0,144.0,0.0,0.8,1.0,0.0,7.0
1111,33.0,0.0,4.0,100.0,246.0,0.0,0.0,150.0,1.0,1.0,2.0,4.0,8.0


We can see that the counterfactuals are similar to the query sample and that they have a flipped prediction. These are the two general properties of counterfactual explanations.

We will now do a small proof of concept of the experiment with logging enabled to demonstrate how it works.

In [8]:
logger.setLevel(logging.DEBUG)
logging.root.setLevel(logging.ERROR)

EXP = CounterfactualMembershipInference(data_heart, numeric_features_heart, outcome_name_heart, random_state=13)
EXP.membership_inference_experiment(num_queries=10, model=DecisionTreeClassifier(random_state=13), model_access=False, threads=1)

logger.setLevel(logging.ERROR)

DEBUG:xai-privacy:Numeric Features: ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
DEBUG:xai-privacy:Categorical Features: ['CA', 'ChestPainType', 'ExerciseAngina', 'FastingBS', 'RestingECG', 'ST_Slope', 'Sex', 'Thal']
DEBUG:xai-privacy:Removed 7 test samples due to unknown category.
DEBUG:xai-privacy:[[ 38.   1.   4. 110. 196.   0.   0. 166.   0.   0.   4.   4.   8.   1.]] taken from training data
DEBUG:xai-privacy:[[ 57.   1.   4. 140. 214.   0.   1. 144.   1.   2.   2.   4.   6.   1.]] taken from test data
DEBUG:xai-privacy:[[ 53.   1.   4. 124. 243.   0.   0. 122.   1.   2.   2.   4.   7.   1.]] taken from training data
DEBUG:xai-privacy:[[ 50.   0.   4. 120. 328.   0.   0. 110.   1.   1.   2.   4.   8.   0.]] taken from test data
DEBUG:xai-privacy:[[ 33.   0.   4. 100. 246.   0.   0. 150.   1.   1.   2.   4.   8.   1.]] taken from training data
DEBUG:xai-privacy:[[ 51.    1.    4.  140.  299.    0.    0.  173.    1.    1.6   1.    0.
    7.    1. ]] taken from test data
D

Total time: 69.43s (training model: 0.03s, training explainer: 0.03s, experiment: 69.37s)
Accuracy: 0.9, precision: 1.0, recall: 0.8


The proof of concept should show that counterfactual membership inference is accurate most of the time. However, some training samples are not recognized because they do not appear as their own counter-counterfactual.

Now we begin executing the actual experiment. We begin by defining the table that will hold the results for all our different experiment variations. Then we execute all variations of the experiment for this dataset. We vary the model between a decision tree, a random forest and a neural network. Each model uses the default configuration of scikit-learn.

In [9]:
dataset_dicts = [data_heart_dict, data_heart_num_dict, data_heart_cat_dict, data_census_dict, data_census_num_dict, data_census_cat_dict]

dt_dict = {'name': 'decision tree', 'model': DecisionTreeClassifier}
rf_dict = {'name': 'random forest', 'model': es.RandomForestClassifier}
nn_dict = {'name': 'neural network', 'model': MLPClassifier}

model_dicts = [dt_dict, rf_dict, nn_dict]

In [10]:
# remove pandas warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [11]:
# This will run the experiment for each dataset and model combination

results = run_all_experiments(CounterfactualMembershipInference, dataset_dicts, model_dicts, random_state=0, num_queries=None, model_access=False, threads=threads, convert_cat_to_str=True, repeat=10)

dataset: heart, model: decision tree (repetition 0)


100%|██████████| 108/108 [02:49<00:00,  1.57s/it]


Total time: 209.53s (training model: 0.03s, training explainer: 0.03s, experiment: 209.47s)
Accuracy: 0.8467153284671532, precision: 1.0, recall: 0.6949152542372882
dataset: heart, model: decision tree (repetition 1)


100%|██████████| 108/108 [03:11<00:00,  1.77s/it]


Total time: 236.99s (training model: 0.03s, training explainer: 0.03s, experiment: 236.93s)
Accuracy: 0.8533007334963325, precision: 1.0, recall: 0.7094430992736077
dataset: heart, model: decision tree (repetition 2)


...

Total time: 4540.02s (training model: 22.64s, training explainer: 0.17s, experiment: 4517.21s)
Accuracy: 0.6884154460719041, precision: 1.0, recall: 0.3769000332852546
dataset: census categorical, model: neural network (repetition 8)


/home/i40/langema/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 2328/2328 [1:00:23<00:00,  1.56s/it]


Total time: 4538.84s (training model: 23.54s, training explainer: 0.19s, experiment: 4515.11s)
Accuracy: 0.6874722592099423, precision: 1.0, recall: 0.3750138688560967
dataset: census categorical, model: neural network (repetition 9)


/home/i40/langema/venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
100%|██████████| 2328/2328 [59:05<00:00,  1.52s/it]


Total time: 4458.79s (training model: 24.12s, training explainer: 0.18s, experiment: 4434.48s)
Accuracy: 0.69661487236404, precision: 1.0, recall: 0.39343170975257963


# Results

The results of all variations of the membership inference experiment with counterfactuals. In every experiment, we executed the membership inference attack on each sample of the training data and each sample of the test data. Both datasets are of equal size and originate from the same source dataset.

Accuracy is the percentage of samples whose membership (true or false) was correctly inferred. An algorithm guessing at random would achieve an accuracy of 50 percent.

Precision is the percentage of predicted training samples that is actually in the training data.

Recall is the percentage of training samples whose membership (true) was correctly inferred.

In [12]:
results

,dataset,model,repetition,accuracy,precision,recall
0,heart,decision tree,0,0.846715,1.0,0.694915
1,heart,decision tree,1,0.853301,1.0,0.709443
2,heart,decision tree,2,0.849265,1.0,0.702179
3,heart,decision tree,3,0.866911,1.0,0.736077
4,heart,decision tree,4,0.857837,1.0,0.716707
...,...,...,...,...,...,...
175,census categorical,neural network,5,0.692111,1.0,0.384223
176,census categorical,neural network,6,0.683390,1.0,0.367025
177,census categorical,neural network,7,0.688415,1.0,0.376900
178,census categorical,neural network,8,0.687472,1.0,0.375014


In [13]:
file_name = 'results/1-6-cf-membership-inference-results'
if DATASET_HALF:
    file_name += '_dataset_size_halved'
results.to_csv(file_name + '.csv', index=False, na_rep='NaN', float_format='%.3f')

# Discussion

In our experiments, membership inference with counterfactuals drawn from the training data has an accuracy between 69% and 85%. Since no false positives can occur, precision is always 100%.

The lower recall of the attack with a neural network and the numeric dataset can be explained by the following observations. Recall increases significantly when one fixes the inbalance in the training dataset between label 0 and 1. Either undersampling the majority class or oversampling the minority class results in membership inference being similarly accurate as for decision trees or random forests.

Why does this change accuracy so much? Let's have a look at the predictions of the three models when using the original inbalanced dataset:

In [14]:
# train models exactly as in the experiment
idx_mid = int(data_heart.shape[0] / 2)

data_train = data_heart.iloc[idx_mid:, :]

x_train = data_train.drop(outcome_name_heart, axis=1)
y_train = data_train[outcome_name_heart]

features = data_heart.drop(outcome_name_heart, axis=1)

model_dt = DecisionTreeClassifier(random_state=0)
model_rf = es.RandomForestClassifier(random_state=0)
model_nn = MLPClassifier(random_state=0)

for model in [model_dt, model_rf, model_nn]:
    model = model.fit(x_train, y_train)
    
# check how often they predict 1/0
pred_dt = model_dt.predict(features)
pred_rf = model_rf.predict(features)
pred_nn = model_nn.predict(features)

print(f'Decision Tree predicted 1 for {np.count_nonzero(pred_dt)} samples.')
print(f'Random Forest predicted 1 for {np.count_nonzero(pred_rf)} samples.')
print(f'Neural Network predicted 1 for {np.count_nonzero(pred_nn)} samples.')
print(f'There is a total of {len(features)} samples.')

Decision Tree predicted 1 for 462 samples.
Random Forest predicted 1 for 450 samples.
Neural Network predicted 1 for 295 samples.
There is a total of 826 samples.


It is clear that the inbalance of the dataset has the biggest negative effect on the neural network and causes it to classify a majority of samples with 0. Therefore, there are fewer counterfactuals in the training data with a prediction of 1. This causes them to be further away from the queries on average and thus reducing the chance of a counter-counterfactual being the same as the query. Therefore, recall of the attack is reduced.

The question remains why the recall is equal for a random forest and decision tree. It turns out that both models perfectly fit to the training data because they are not limited in complexity. Therefore they also give exactly the same predictions for the training data and thus receive the same counterfactuals. They do not give the same predictions for the test data, but this does not matter, because this attack will always reject samples from outside the training data regardless of the model (simply because no counterfactual can be shown that is equal to the given sample).